In [2]:
import pandas as pd

# IPCA

In [4]:
ipca = (pd.read_csv("/Users/matheus.facure/Downloads/20221227034122.csv", sep=";", )
        .T
        .assign(date=pd.date_range("1994-07-01", "2022-12-01", freq="M")+pd.offsets.MonthBegin(-1))
        .rename(columns={"Brasil": "ipca"})
        .assign(ipca = lambda d: d["ipca"]/100)
        .query("date>='1995-01-01'")
        .reset_index(drop=True)
       )


ipca.to_csv("ipca.csv", index=False)
ipca.tail()

,ipca,date
330,-0.0068,2022-07-01
331,-0.0036,2022-08-01
332,-0.0029,2022-09-01
333,0.0059,2022-10-01
334,0.0041,2022-11-01


# Dolar

In [21]:
dolar1 = pd.read_csv("/Users/matheus.facure/Downloads/USD_BRL Dados Históricos.csv")
dolar2 = pd.read_csv("/Users/matheus.facure/Downloads/USD_BRL Dados Históricos (1).csv")
dolar = (pd.concat([dolar1, dolar2])
         .drop_duplicates()
         [["Data", "Último"]]
         .rename(columns={"Data":"date", "Último":"dolar"})
         .assign(date=lambda d: pd.to_datetime(d["date"], format="%d.%m.%Y"),
                 dolar=lambda d: pd.to_numeric(d["dolar"].str.replace(",", ".")))
         .sort_values("date"))

dolar.head()

,date,dolar
4999,1994-01-27,0.1609
4998,1994-01-28,0.1637
4997,1994-01-31,0.1668
4996,1994-02-01,0.1700
4995,1994-02-02,0.1732


In [22]:
dolar.to_csv("brl_usd.csv", index=False)

# SELIC

In [23]:
selic_ = pd.read_html("http://www.ipeadata.gov.br/exibeserie.aspx?serid=38402", match='Taxa de juros nominal - Over / Selic')[1]

In [24]:
def anual_to_monthly_rate(r):
    return ((1 + r)**(1/12)) - 1


In [25]:
selic = (selic_
         .drop(index=[0])
         .rename(columns={0:"date", 1:"selic"})
         .assign(selic_aa=lambda d: pd.to_numeric(d["selic"].str.replace("\.", "").str.replace(",", ""))/10000,
                 date=lambda d: pd.to_datetime(d["date"]+".01", format="%Y.%m.%d"))
         .query("date>='1994-07-01'")
         .assign(selic = lambda d: anual_to_monthly_rate(d["selic_aa"])))

selic.head()

/Users/matheus.facure/miniforge3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


,date,selic,selic_aa
247,1994-07-01,0.068701,1.2196
248,1994-08-01,0.041699,0.6327
249,1994-09-01,0.038300,0.5699
250,1994-10-01,0.036198,0.5322
251,1994-11-01,0.040699,0.6140


In [26]:
selic.to_csv("selic.csv", index=False)

# SP500

In [27]:
sp500_1 = pd.read_csv("/Users/matheus.facure/Downloads/S&P 500 Historical Data.csv")
sp500_2 = pd.read_csv("/Users/matheus.facure/Downloads/S&P 500 Historical Data (1).csv")

sp500 = (pd.concat([sp500_1, sp500_2]).drop_duplicates()
         .assign(date=lambda d: pd.to_datetime(d["Date"], format="%m/%d/%Y"),
                 sp500=lambda d: pd.to_numeric(d["Price"].str.replace(",", "")))
         [["date", "sp500"]]
         .query("date>='1994-01-01'")
         .sort_values("date"))

sp500.head()

,date,sp500
4999,1994-01-03,465.4
4998,1994-01-04,466.9
4997,1994-01-05,467.6
4996,1994-01-06,467.1
4995,1994-01-07,469.9


In [28]:
sp500.to_csv("sp500.csv", index=False)